# An HMM demo for Turing

In [22]:
using Turing, Distributions
using PyPlot, PyCall
@pyimport matplotlib.cm as cm
@pyimport matplotlib.animation as animation
@pyimport numpy as np
@pyimport IPython.display as D

Load transition matrix T and observations obs

In [23]:
include(Pkg.dir("Turing")*"/notebooks/data/hmmdemo.data.jl");

Define the model

In [24]:
K = 5
N = 201
initial = fill(1.0 / K, K)
means = (collect(1.0:K)*2-K)*2

@model hmmdemo begin
    states = tzeros(Int,N)
    # T = TArray{Array{Float64,}}
    
    # Prior over T
    # for i=1:K, @assume T[i,:] ~ Dirichlet(ones(K)./K); end
    
    @assume states[1] ~ Categorical(initial)
    for i = 2:N
        @assume states[i] ~ Categorical(vec(T[states[i-1],:]))
        @observe obs[i] ~ Normal(means[states[i]], 4)
    end
    @predict states
end

hmmdemo (generic function with 1 method)

Run the sampler, collect results

In [25]:
chain = sample(hmmdemo, PG(100,100));

[PG]: Iter 10 out of 100
[PG]: Iter 20 out of 100
[PG]: Iter 30 out of 100
[PG]: Iter 40 out of 100
[PG]: Iter 50 out of 100
[PG]: Iter 60 out of 100
[PG]: Iter 70 out of 100
[PG]: Iter 80 out of 100
[PG]: Iter 90 out of 100
[PG]: Iter 100 out of 100


Print the 1st sample trajectory

In [26]:
chain.value[1].value[:states]'

1x201 Array{Int64,2}:
 1  3  4  4  2  4  2  5  5  5  1  5  1  …  1  4  2  2  4  4  4  4  4  4  4  2

Plot samples

In [27]:
fig, ax = plt[:subplots]();
ax[:set_xlim](( 0, 210))
ax[:set_ylim]((-20, 20))
line, = ax[:plot]([], [], lw=2)

# initialization function: plot the background of each frame
function init()
    ax[:plot](1:N, obs, "rs")
    return (line,)
end

# animation function. This is called sequentially
function animate(i)
    n = mod(i,length(chain.value))
    line[:set_data](1:N, means[chain.value[n+1].value[:states]])
    return (line,)
end

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=200, blit=true)

plt[:close]()


In [21]:
D.HTML(anim[:to_html5_video]())

PyObject <IPython.core.display.HTML object>